Approximating CDFs for Nash distribution

In [252]:
using Random
using DataFrames
using CSV
using Statistics
using Printf
using Distributions
using LinearAlgebra

struct Fighter
    name::String
    h::Int16
    f::Int16
    l::Int16
    t::Int16
end

const MXS = 10000
const MXV = 200
const MNV = 10

function random_fighter(h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    flag = true
    while flag
        h = h0 + rand(0:(MXV - h0), 1)[1]
        f = f0 + rand(0:(MXV - f0), 1)[1]
        l = l0 + rand(0:(MXV - l0), 1)[1]
        t = t0 + rand(0:(MXV - t0), 1)[1]
        mm = minimum([h, f])
        budget = h*f+h*l+f*t
        if budget <= MXS && budget + mm > MXS
            flag = false
            return Fighter("No Name", h, f, l, t)
        end
    end
end

function eval_battle(a::Fighter, b::Fighter)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    if a_finds && b_finds
        if a.t > b.t
            return 1
        end
        if a.t < b.t
            return -1
        end
        if a.t == b.t
            if a.l > b.l
                return 1
            end
            if a.l < b.l
                return -1
            end
            if a.l == b.l
                if a.h > b.h
                    return 1
                end
                if a.h < b.h
                    return -1
                end
                if a.h == b.h
                    return 0
                end
            end
        end
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        if a.l > b.l
            return 1
        end
        if a.l < b.l
            return -1
        end
        if a.l == b.l
            if a.t > b.t
                return 1
            end
            if a.t < b.t
                return -1
            end
            if a.t == b.t
                if a.f > b.f
                    return 1
                end
                if a.f < b.f
                    return -1
                end
                if a.f == b.f
                    return 0
                end
            end
        end
    end
end

function random_tournament_winner(c::Int, f::Function = random_fighter)::Fighter
    if c==0
        return f()
    end
    a = random_tournament_winner(c-1, f)
    b = random_tournament_winner(c-1, f)
    res = eval_battle(a, b)
    if res == 1
        return a
    else
        return b
    end
end


tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)

function random_name_and_stat()::Fighter
    vp = [0, 0, 0, 0]
    nametype = rand(1:5)
    name = ""
    if nametype == 1 || nametype == 2
        nm = rand(names)
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        if nametype == 1
            name = string(nm, " the ", adj)
        end
        if nametype == 2
            name = string(adj, " ", nm)
        end
    end
    if nametype == 3 || nametype == 4
        nm = rand(names)
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        if nametype == 3
            name = string(nm, " the ", noun)
        end
        if nametype == 4
            name = string(noun, " ", nm)
        end
    end
    if nametype == 5
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        name = string(adj, " ", noun)
    end
    Fighter(name, vp[1], vp[2], vp[3], vp[4])
end

function rand_rename(a::Fighter, n_tries::Int = 10)::Fighter
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.t^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.t * b.t)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Fighter(best_b.name, a.h, a.f, a.l, a.t)
end

function eval_battle_list(a::Fighter, bs::Array{Fighter})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Fighter, bs::Array{Fighter}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end


function pick_best(as::Array{Fighter}, bs::Array{Fighter})::Fighter
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Fighter}, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function pick_best_rdmly_g(g::Function, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = g()
    for ii in 1:ntries
        f = g()
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function fighters_to_df(as::Array{Fighter})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ts = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ts[ii] = as[ii].t    
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, t = ts)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Fighter}, bs::Array{Fighter})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

function compare_generator(f1, f2, limit)
    a_i = 1
    b_i = 1
    f_a = f1()
    f_b = f2()
    while (a_i < limit) && (b_i < limit)
        res = eval_battle(f_a, f_b)
        if res != -1
            b_i = b_i + 1
            f_b = f2()
        end
        if res != 1
            a_i = a_i + 1
            f_a = f1()
        end
    end
    return (a_i/limit, b_i/limit)
end

function random_team(f::Function, team_size::Int)::Array{Fighter}
    team = Array{Fighter}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [188]:
@time library = random_team(() -> rand_rename(random_fighter()), 100000)

 44.897691 seconds (1.00 G allocations: 62.550 GiB, 14.68% gc time, 0.39% compilation time)


100000-element Vector{Fighter}:
 Fighter("Rayvion the Yak", 39, 53, 77, 93)
 Fighter("Hot Jennika", 22, 37, 123, 175)
 Fighter("Herlinda the Strawberry", 41, 71, 146, 15)
 Fighter("Sassy Monkey", 115, 32, 48, 25)
 Fighter("Popular Archer", 43, 69, 73, 56)
 Fighter("Aubreyanna the Monkey", 71, 81, 18, 36)
 Fighter("Herminio the Scorpion", 45, 148, 15, 18)
 Fighter("Aggressive Joker", 31, 37, 62, 187)
 Fighter("Popular Mushroom", 67, 16, 109, 101)
 Fighter("Ruqayyah the Dizzy", 56, 20, 91, 189)
 Fighter("Brutal Xerox", 80, 34, 11, 188)
 Fighter("Creative Gamer", 52, 52, 63, 77)
 Fighter("Hunter Aunystee", 38, 62, 42, 97)
 ⋮
 Fighter("Boot Banks", 88, 43, 40, 62)
 Fighter("Wilda the Yoyo", 137, 23, 45, 29)
 Fighter("Jaecion the Daring", 20, 31, 182, 185)
 Fighter("Manta Londynn", 61, 29, 71, 134)
 Fighter("Skiler the Rhino", 33, 55, 16, 139)
 Fighter("Amelya the Trampoline", 30, 22, 176, 184)
 Fighter("Soldier Amin", 33, 59, 77, 93)
 Fighter("Eiliyah the Frozen", 73, 50, 50, 54)
 Fighter(

In [196]:
function mcmc_fighter_v1(; budget::Int = MXS, h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV, nits::Int = 100)::Fighter
    name = "MCMC v.1"
    h = h0
    f = f0
    l = l0
    t = t0
    for iter in 1:nits
        # randomly decrease one stat
        switch = rand(1:4)
        if switch == 1
            h = rand(h0:h)
        end
        if switch == 2
            f = rand(f0:f)
        end
        if switch == 3
            l = rand(l0:l)
        end
        if switch == 4
            t = rand(t0:t)
        end
        # increase another stat
        switch2 = rand(1:4)
        while switch2 == switch
            switch2 = rand(1:4)
        end
        if switch2 == 1
            h = min(convert(Int, trunc((budget - (f * t))/(f + l))), MXV)
        end
        if switch2 == 2
            f = min(convert(Int, trunc((budget - (h * l))/(h + t))), MXV)
        end
        if switch2 == 3
            l = min(convert(Int, trunc((budget - (h * f + f * t))/h)), MXV)
        end
        if switch2 == 4
            t = min(convert(Int, trunc((budget - (h * f + h * l))/f)), MXV)
        end        
    end
    return Fighter(name, h, f, l, t)
end


mcmc_fighter_v1 (generic function with 7 methods)

In [328]:
@time library_m = random_team(() -> rand_rename(mcmc_fighter_v1()), 100000)
0

  1.696633 seconds (11.32 M allocations: 496.480 MiB, 4.87% gc time, 0.66% compilation time)


0

In [4]:
#tab = fighters_to_df(library[1:20])

In [5]:
# v = tab.t
# for i in 1:20
#     a = round(v[i]/10)
#     if a > 9
#         print(", ")
#     else
#         print(",  ")
#     end
#     print(convert(Int, a))
# end

In [7]:
budget = MXS
budget2 = convert(Int, trunc(0.7 * budget))
h0 = MNV
f0 = MNV
l0 = MNV
t0 = MNV
nits = 5
maxstep = 10

10

In [148]:
opt = rand(1:20)
name = string("MCMC v.2-", @sprintf("%i", opt))
#       1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20
hrs = [ 4,  2,  6, 14,  6, 16,  3,  8,  2,  6,  2,  6, 10, 17,  3,  4,  2,  4,  8, 14]
frs = [ 5, 11,  7,  3,  6,  2,  6,  4, 18,  2,  5,  3,  6,  2,  7,  5,  8,  3,  5,  2]
lrs = [14,  6,  1,  2,  1,  3,  7,  4,  5, 14,  6, 13,  2,  1, 17,  9, 19, 11,  5,  3]
trs = [ 5,  5,  7,  8,  8,  7, 11,  7,  3,  4, 14,  4,  4, 15,  4, 11,  6, 18,  4, 11]
mult = sqrt(budget/100)
h = max(h0, convert(Int, trunc(mult * hrs[opt])))
f = max(f0, convert(Int, trunc(mult * frs[opt])))
l = max(l0, convert(Int, trunc(mult * lrs[opt])))
t = max(t0, convert(Int, trunc(mult * trs[opt])))
name,h,f,l,t

("MCMC v.2-13", 100, 60, 20, 40)

In [158]:
if (h*f + h*l + t*f > budget2)
    switch = rand(1:4)
    step = rand(1:maxstep)
    if switch == 1
        h = max(h0, h - step)
    end
    if switch == 2
        f = max(f0, f - step)
    end
    if switch == 3
        l = max(l0, l - step)
    end
    if switch == 4
        t = max(t0, t - step)
    end
    print((name,h,f,l,t))
end

In [176]:
if ((h*f + h*l + t*f > budget) || (h*f + h*l + t*f + min(h, f) <= budget))
    # increase or decrease another stat
    switch = rand(1:4)
    if h*f + h*l + t*f > budget
        step = rand(1:maxstep)
    end
    if h*f + h*l + t*f + min(h, f) <= budget
        step = rand(-maxstep:-1)
    end
    if switch == 1
        h = max(h0, h - step)
    end
    if switch == 2
        f = max(f0, f - step)
    end
    if switch == 3
        l = max(l0, l - step)
    end
    if switch == 4
        t = max(t0, t - step)
    end
    println((switch,step))
    print((name,h,f,l,t))
end


In [201]:
function mcmc_fighter_v2(; budget::Int = MXS, budget2::Int = convert(Int, trunc(0.7 * budget)),
        h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV, 
        nits::Int = 2, maxstep::Int = 10)::Fighter
    opt = rand(1:20)
    name = string("MCMC v.2-", @sprintf("%i", opt))
    #       1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20
    hrs = [ 4,  2,  6, 14,  6, 16,  3,  8,  2,  6,  2,  6, 10, 17,  3,  4,  2,  4,  8, 14]
    frs = [ 5, 11,  7,  3,  6,  2,  6,  4, 18,  2,  5,  3,  6,  2,  7,  5,  8,  3,  5,  2]
    lrs = [14,  6,  1,  2,  1,  3,  7,  4,  5, 14,  6, 13,  2,  1, 17,  9, 19, 11,  5,  3]
    trs = [ 5,  5,  7,  8,  8,  7, 11,  7,  3,  4, 14,  4,  4, 15,  4, 11,  6, 18,  4, 11]
    mult = sqrt(budget/100)
    h = max(h0, convert(Int, trunc(mult * hrs[opt])))
    f = max(f0, convert(Int, trunc(mult * frs[opt])))
    l = max(l0, convert(Int, trunc(mult * lrs[opt])))
    t = max(t0, convert(Int, trunc(mult * trs[opt])))
    for iter in 1:nits
        while (h*f + h*l + t*f > budget2)
            switch = rand(1:4)
            step = rand(1:maxstep)
            if switch == 1
                h = max(h0, h - step)
            end
            if switch == 2
                f = max(f0, f - step)
            end
            if switch == 3
                l = max(l0, l - step)
            end
            if switch == 4
                t = max(t0, t - step)
            end
        end
        while (h*f + h*l + t*f > budget) || (h*f + h*l + t*f + min(h, f) <= budget)
            # increase or decrease another stat
            switch = rand(1:4)
            if h*f + h*l + t*f > budget
                step = rand(1:maxstep)
            end
            if h*f + h*l + t*f + min(h, f) <= budget
                step = rand(-maxstep:-1)
            end
            if switch == 1
                h = max(h0, h - step)
            end
            if switch == 2
                f = max(f0, f - step)
            end
            if switch == 3
                l = max(l0, l - step)
            end
            if switch == 4
                t = max(t0, t - step)
            end
        end
    end
    return Fighter(name, h, f, l, t)
end

mcmc_fighter_v2()

Fighter("MCMC v.2-2", 19, 134, 131, 37)

In [178]:
f = mcmc_fighter_v2()
println(f.h * f.f + f.h * f.l + f.f * f.t)
println(f)

9992
Fighter("MCMC v.2-18", 13, 48, 152, 154)


In [180]:
function nashprox(g, size, start_size, nt)
    nash_env = Array{Fighter}(undef, size)
    nash_env[1:start_size] = random_team(g, start_size)
    for j in (start_size+1):size
        ff = pick_best_rdmly_g(g, nash_env[1:(j-1)], nt)
        nash_env[j] = ff
    end
    return nash_env
end


function nashprox2(g, size, start_size, nt, ns)
    nash_env = Array{Fighter}(undef, size)
    nash_env[1:start_size] = random_team(g, start_size)
    for j in (start_size+1):size
        if j < ns
            ff = pick_best_rdmly_g(g, nash_env[1:(j-1)], nt)
        else
            ff = pick_best_rdmly_g(g, randsubseq(nash_env[1:(j-1)], ns/j), nt)
        end
        nash_env[j] = ff
    end
    return nash_env
end




nashprox2 (generic function with 1 method)

In [85]:
@time nash_env0 = nashprox(() -> rand(library), 10000, 10, 99)
0

 32.791842 seconds (1.04 M allocations: 795.237 MiB, 0.29% gc time, 0.11% compilation time)


0

In [87]:
@time nash_env0a = nashprox(() -> rand(library), 20000, 10, 20)
0

 30.300878 seconds (488.01 k allocations: 2.996 GiB, 1.06% gc time, 0.14% compilation time)


0

In [96]:
@time nash_env0b = nashprox2(() -> rand(library), 80000, 10, 20, 1000)
0

 32.007045 seconds (2.09 M allocations: 49.118 GiB, 17.66% gc time, 0.12% compilation time)


0

In [97]:
@time nash_env1a = nashprox(mcmc_fighter_v1, 15000, 10, 40)
0

 29.265473 seconds (43.96 k allocations: 1.678 GiB, 0.84% gc time)


0

In [233]:
@time nash_env1b = nashprox2(mcmc_fighter_v1, 10000, 10, 40, 1000)
0

  4.319508 seconds (46.96 k allocations: 924.495 MiB, 5.66% gc time)


0

In [193]:
@time nash_env2b = nashprox2(mcmc_fighter_v2, 10000, 10, 40, 1000)
0

  4.124739 seconds (2.91 M allocations: 1.290 GiB, 6.55% gc time)


0

In [190]:
function evaluate_nash(nash_env, library)
    counter = pick_best(library, nash_env)
    println(counter)
    return eval_battle_list(counter, nash_env)/length(nash_env)
end

evaluate_nash (generic function with 2 methods)

In [102]:
println(evaluate_nash(nash_env0, library))
println(evaluate_nash(nash_env0a, library))
println(evaluate_nash(nash_env0b, library))
println(evaluate_nash(nash_env1a, library))
println(evaluate_nash(nash_env1b, library))


Fighter("Tyro Aroush", 69, 10, 132, 20)
0.0668
Fighter("Ahsan the Zinc", 10, 67, 15, 137)
0.0704
Fighter("Xile the Ladder", 10, 79, 12, 115)
0.098675
Fighter("Brutal Rayland", 10, 65, 12, 142)
0.032
Fighter("Accurate Lawyer", 47, 119, 10, 33)
0.03286666666666667


In [200]:
println(evaluate_nash(nash_env1b, library))
println(evaluate_nash(nash_env1c, library))
println(evaluate_nash(nash_env2b, library))

Fighter("Manta Dashiell", 47, 18, 140, 143)
0.038
Fighter("Helicopter Ariv", 10, 93, 14, 96)
0.0432
Fighter("Mosquito Fidencio", 65, 121, 10, 12)
0.076


In [199]:
@time nash_env1c = nashprox2(() -> mcmc_fighter_v1(nits = 20), 10000, 10, 40, 1000)
0

  3.402281 seconds (95.52 k allocations: 927.842 MiB, 5.91% gc time, 1.34% compilation time)


0

In [203]:
@time nash_env1c = nashprox2(() -> mcmc_fighter_v2(nits = 20), 10000, 10, 40, 1000)
0

 12.244178 seconds (2.99 M allocations: 1.295 GiB, 2.24% gc time, 0.41% compilation time)


0

In [215]:
@time nash_env1d = nashprox2(() -> mcmc_fighter_v1(), 1000, 10, 40, 1000)
0

  0.294002 seconds (11.23 k allocations: 8.514 MiB, 12.72% compilation time)


0

In [216]:
println(evaluate_nash(nash_env1d, library[1:1000]))

Fighter("Hungry Mylee", 34, 153, 19, 27)
0.076


## Greedy optimization

In [237]:
nash_env = random_team(mcmc_fighter_v1, 20000)
g = mcmc_fighter_v1
0

0

In [238]:
function evaluate_nash(nash_env, library)
    counter = pick_best(library, nash_env)
    #println(counter)
    return eval_battle_list(counter, nash_env)/length(nash_env)
end

nits = 5
nlib = 20
for iter in 1:nits
    lib = random_team(mcmc_fighter_v1, nlib)
    lib_i = 1
    for ind in 1:length(nash_env)
        lib[lib_i] = g()
        lib_i = lib_i + 1
        if lib_i > nlib
            lib_i = 1
        end
        score = evaluate_nash(nash_env, lib)
        newf = g()
        oldf = nash_env[ind]
        nash_env[ind] = newf
        newscore = evaluate_nash(nash_env, lib)
        if newscore > score
            nash_env[ind] = oldf
        end
    end
    print(evaluate_nash(nash_env, library))
    print(" ")
end

0.4957 0.441 0.4185 

LoadError: InterruptException:

In [231]:
# function evaluate_nash(nash_env, library)
#     counter = pick_best(library, nash_env)
#     #println(counter)
#     return eval_battle_list(counter, nash_env)/length(nash_env)
# end

# nits = 5
# nlib = 200
# score = evaluate_nash(nash_env, library[1:nlib])
# for iter in 1:nits
#     for ind in 1:length(nash_env)
#         newf = g()
#         oldf = nash_env[ind]
#         nash_env[ind] = newf
#         newscore = evaluate_nash(nash_env, library[1:nlib])
#         if newscore <= score
#             score = newscore
#         else
#             nash_env[ind] = oldf
#         end
#     end
#     print(score)
#     print(" ")
# end

0.023 0.0228 0.0224 0.0224 0.022 

In [232]:
function evaluate_nash(nash_env, library)
    counter = pick_best(library, nash_env)
    println(counter)
    return eval_battle_list(counter, nash_env)/length(nash_env)
end

evaluate_nash(nash_env, library)

Fighter("Scissors Gionna", 19, 79, 93, 85)


0.0578

In [234]:
println(evaluate_nash(nash_env1b, library))

Fighter("Xquisite Buxton", 29, 32, 147, 150)
0.0532


## Nash prob dist

In [302]:
@time nash_env = nashprox2(() -> rand_rename(mcmc_fighter_v1()), 10000, 10, 40, 1000)
0

  9.538125 seconds (46.52 M allocations: 2.890 GiB, 4.20% gc time, 1.67% compilation time: 18% of which was recompilation)


0

In [313]:
w = [1.0/length(nash_env) for n=1:length(nash_env)]
eps = 0.01/length(nash_env)

1.0e-6

In [325]:
nits = 100
for i in 1:nits
    scores = [eval_battle_list2(ff, nash_env, w) for ff in nash_env]
    #w[scores .< 0] = w[scores .< 0] .- eps
    w[scores .> 0] = w[scores .> 0] .+ eps
    w[w .< 0] .= 0
    w = w./sum(w)
    if mod1(i, 20)==1
        println((maximum(scores), maximum(w * length(nash_env)), minimum(w * length(nash_env))))
        mxv, mxi = findmax(w)
        println(nash_env[mxi])
        mxv, mxi = findmax(scores)
        println(nash_env[mxi])
    end
end

(0.04629228102700389, 2.635605816677576, 0.021470306536401824)
Fighter("Furious Scissors", 27, 29, 165, 164)
Fighter("Furious Candle", 37, 20, 162, 163)
(0.043794791708530616, 2.618354846660965, 0.019769739338505198)
Fighter("Furious Scissors", 27, 29, 165, 164)
Fighter("Iron Dahila", 10, 58, 11, 160)
(0.04840988321066415, 2.626364016574821, 0.018375578423944317)
Fighter("Furious Scissors", 27, 29, 165, 164)
Fighter("Turbo Graciana", 159, 27, 34, 10)
(0.040008346546361796, 2.6375924318112274, 0.01710710351879227)
Fighter("Furious Scissors", 27, 29, 165, 164)
Fighter("Diamond Tremani", 71, 10, 129, 13)
(0.0505953774055369, 2.624813097814121, 0.015780153770220882)
Fighter("Furious Scissors", 27, 29, 165, 164)
Fighter("Happy Politician", 14, 45, 151, 161)


In [326]:
lib_scores0 = [eval_battle_list2(ff, nash_env, [1.0/length(nash_env) for n=1:length(nash_env)]) for ff in library]
lib_scores = [eval_battle_list2(ff, nash_env, w) for ff in library]
maximum(lib_scores0), maximum(lib_scores)

(0.04920000000000038, 0.05215557266997716)

In [329]:
lib_scores0 = [eval_battle_list2(ff, nash_env, [1.0/length(nash_env) for n=1:length(nash_env)]) for ff in library_m]
lib_scores = [eval_battle_list2(ff, nash_env, w) for ff in library_m]
maximum(lib_scores0), maximum(lib_scores)

(0.048600000000000365, 0.05431412649993631)